## Generate CFs from WRF Data

This file creates annual capacty factor files for the GCM in the folder it is pointed to, running wall time abt 20-25 minutes per year 

In [1]:
gcm = 'ec-earth3-veg_r1i1p1f1_ssp370_bc' 

In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import time
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

In [3]:
from scipy import interpolate
#specifiying power curves

powercurves = pd.read_excel('/nfs/turbo/seas-mtcraig-climate/Martha_Research/EECS Project/wind_turbine_power_curves.xlsx',engine='openpyxl')

powercurve_IECI = interpolate.interp1d(powercurves['Wind Speed'],powercurves['Composite IEC Class I'], bounds_error=False, fill_value=0)
powercurve_IECII = interpolate.interp1d(powercurves['Wind Speed'],powercurves['Composite IEC Class II'])
powercurve_IECIII = interpolate.interp1d(powercurves['Wind Speed'],powercurves['Composite IEC Class III'])

#wind functions
from wind_generation import calc_wind_power
from calc_pv_potential import calc_pv_potential
from calc_speed import calc_speed

#### Create Annaul CF Files for a GCM 
(change file path when run for different GCMs) 

In [4]:
# Create a list to store the processed datasets that will be combined
processed_dataset_list = []

years = [2061]

# Walk through the main folder
for YEAR in years:
        loop_start_time = time.time()
        # Construct the full file path
        file_path = f"/nfs/turbo/seas-mtcraig-climate/WRFDownscaled/{gcm}/{YEAR}/regrid_{YEAR}_ssp370_d02.nc"
        #Open the dataset 
        print(YEAR)
        ds = xr.open_dataset(file_path)

        #realign times 
        start_time = pd.Timestamp(f'{YEAR}-09-01 00:00')
        times = pd.date_range(start=start_time, periods=len(ds.Times), freq='h')
        ds['Times'] = times
        #remove last hour of data because there is one hour of the next september 
        ds = ds.isel(Times=slice(None, -1))

        #setting up wind data
        wind_data = {}
        #getting 100m wind speeds
        wind_data['u100'] = (ds.U10*(100/10)**(1/7)).rename('u100')
        wind_data['v100'] = (ds.V10*(100/10)**(1/7)).rename('v100')
        #assigning attributes
        wind_data['u100'].attrs.update(units= 'm s**-1', long_name = '100 metre U wind component')
        wind_data['v100'].attrs.update(units= 'm s**-1', long_name = '100 metre V wind component')
        uv100 = xr.merge([wind_data['u100'], wind_data['v100']])
        #combing data to feed into calc wind power function 
        wind_speed = xr.map_blocks(calc_speed,uv100,args=['u100','v100']).compute()
        wind_data['ps'] = ds.PSFC.rename('ps')
        wind_data['tas'] = ds.T2.rename('tas')
        wind_data['huss'] = ds.Q2.rename('huss')
        wind_dset = xr.merge([wind_speed.to_dataset(name='100mWind'), wind_data['ps'], wind_data['tas'], wind_data['huss']])
        wind_power = calc_wind_power(wind_dset,powercurve_IECI).compute()
        WindPotential = wind_power/1500
        WindPotential = WindPotential.rename('Wind_CF')
        WindPotential = WindPotential.compute()
        ds = ds.assign(Wind_CF = WindPotential)

        #calculating solar cfs 
        solar_data = {}
        solar_data['rsds'] = ds.SWDNB.rename('rsds')
        #getting 10m wind speeds
        wind_data['u10'] = ds.U10.rename('u10')
        wind_data['v10'] = ds.V10.rename('v10')
        #assigning attributes
        wind_data['u10'].attrs.update(units= 'm s**-1', long_name = '10 metre U wind component')
        wind_data['v10'].attrs.update(units= 'm s**-1', long_name = '10 metre V wind component')
        uv10 = xr.merge([wind_data['u10'], wind_data['v10']])
        #computing net wind speed
        wind_speed10 = xr.map_blocks(calc_speed,uv10,args=['u10','v10']).compute()
        #temp 
        solar_data['tas'] = ds.T2.rename('tas')
        solar_dset = xr.merge([wind_speed10.to_dataset(name='surfWind'), solar_data['rsds'], solar_data['tas']])
        #calculations
        pv_pot = calc_pv_potential(solar_dset)
        pv_pot = pv_pot.rename('Solar_CF')
        pv_pot.attrs['units'] = 'None'
        pv_pot.attrs['SS description'] = 'This is the solar potential of a site. Multiplying this by 1MW will give the ' \
                                         'electricity generation from a 1MW plant at different times and locations. '
        ds = ds.assign(Solar_CF = pv_pot)

        #drop other can capacity factors 
        ds = ds.drop_vars(['Q2', 'T2', 'PSFC', 'U10', 'V10', 'SWDNB', 'RUNSF', 'RUNSB'])

        # Append the dataset with calculated variable to the list
        processed_dataset_list.append(ds)

        # Save the combined dataset to a new file
        output_filename = f'/nfs/turbo/seas-mtcraig-climate/WRFDownscaled/{gcm}/Annual_Solar_Wind/Solar_Wind_CFs_{YEAR}.nc'  # Replace with your desired output file path
        ds.to_netcdf(output_filename)
        
        print(f"File saved to {output_filename}")
        loop_end_time = time.time()
        elapsed_time = loop_end_time - loop_start_time
        print(elapsed_time) 

2061
File saved to /nfs/turbo/seas-mtcraig-climate/WRFDownscaled/ec-earth3-veg_r1i1p1f1_ssp370_bc/Annual_Solar_Wind/Solar_Wind_CFs_2061.nc
2765.38996219635


#### Time Check for Process and Graphing Check if Needed

In [5]:
%%time
ds = xr.open_dataset(f"/nfs/turbo/seas-mtcraig-climate/WRFDownscaled/{gcm}/2019/regrid_2019_ssp370_d02.nc")

#realign times 
start_time = pd.Timestamp(f'2019-09-01 00:00')
times = pd.date_range(start=start_time, periods=len(ds.Times), freq='h')
ds['Times'] = times
#remove last hour of data because there is one hour of the next september 
ds = ds.isel(Times=slice(None, -1))

#setting up wind data
wind_data = {}
#getting 100m wind speeds
wind_data['u100'] = (ds.U10*(100/10)**(1/7)).rename('u100')
wind_data['v100'] = (ds.V10*(100/10)**(1/7)).rename('v100')
#assigning attributes
wind_data['u100'].attrs.update(units= 'm s**-1', long_name = '100 metre U wind component')
wind_data['v100'].attrs.update(units= 'm s**-1', long_name = '100 metre V wind component')
uv100 = xr.merge([wind_data['u100'], wind_data['v100']])
#combing data to feed into calc wind power function 
wind_speed = xr.map_blocks(calc_speed,uv100,args=['u100','v100']).compute()
wind_data['ps'] = ds.PSFC.rename('ps')
wind_data['tas'] = ds.T2.rename('tas')
wind_data['huss'] = ds.Q2.rename('huss')
wind_dset = xr.merge([wind_speed.to_dataset(name='100mWind'), wind_data['ps'], wind_data['tas'], wind_data['huss']])
wind_power = calc_wind_power(wind_dset,powercurve_IECI).compute()
WindPotential = wind_power/1500
WindPotential = WindPotential.rename('Wind_CF')
WindPotential = WindPotential.compute()
ds = ds.assign(Wind_CF = WindPotential)
    
#calculating solar cfs 
solar_data = {}
solar_data['rsds'] = ds.SWDNB.rename('rsds')
#getting 10m wind speeds
wind_data['u10'] = ds.U10.rename('u10')
wind_data['v10'] = ds.V10.rename('v10')
#assigning attributes
wind_data['u10'].attrs.update(units= 'm s**-1', long_name = '10 metre U wind component')
wind_data['v10'].attrs.update(units= 'm s**-1', long_name = '10 metre V wind component')
uv10 = xr.merge([wind_data['u10'], wind_data['v10']])
#computing net wind speed
wind_speed10 = xr.map_blocks(calc_speed,uv10,args=['u10','v10']).compute()
#temp 
solar_data['tas'] = ds.T2.rename('tas')
solar_dset = xr.merge([wind_speed10.to_dataset(name='surfWind'), solar_data['rsds'], solar_data['tas']])
#calculations
pv_pot = calc_pv_potential(solar_dset)
pv_pot = pv_pot.rename('Solar_CF')
pv_pot.attrs['units'] = 'None'
pv_pot.attrs['SS description'] = 'This is the solar potential of a site. Multiplying this by 1MW will give the ' \
                                 'electricity generation from a 1MW plant at different times and locations. '
ds = ds.assign(Solar_CF = pv_pot)
ds


FileNotFoundError: [Errno 2] No such file or directory: '/nfs/turbo/seas-mtcraig-climate/WRFDownscaled/ec-earth3-veg_r1i1p1f1_ssp370_bc/2019/regrid_2019_ssp370_d02.nc'

In [ ]:
windcf_slice = ds.windcf.isel(Times=1)

# Set up the map projection and the figure using Plate Carrée
proj = ccrs.PlateCarree()
fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=proj)

# Add state borders from Natural Earth data
states = cfeature.NaturalEarthFeature(category='cultural',
                                      name='admin_1_states_provinces_lines',
                                      scale='50m',
                                      facecolor='none')
ax.add_feature(states, edgecolor='black')

# Plot the data on the map with the same Plate Carrée projection
windcf_slice.plot(ax=ax, transform=ccrs.PlateCarree())

# Set the extent (adjust to your dataset's extents if needed)
ax.set_extent([-125, -66.5, 20, 50], crs=ccrs.PlateCarree())

# Add coastlines and borders for context
ax.coastlines('10m', linewidth=0.8)
ax.add_feature(cfeature.BORDERS, linestyle='-', edgecolor='black')

plt.show()